## Week 2 - Part 2 Solutions
- September 2023
- [https://https://github.com/tisimpson/bioinformatics1](https://github.com/tisimpson/bioinformatics1)
- [ian.simpson@ed.ac.uk](mailto:ian.simpson@ed.ac.uk)

### Challenge 1 - Finding Genes with NCBI-Entrez
Using either the Entrez website to search and/or using what you've learned about BioPython's abilities to query NCBI services retreive entries for a gene called Nrg1.
- How many different gene entries are there for this gene in NCBI databases?
- What is the full name of this gene?
- What kind of protein does this gene encode?

In [ ]:
from Bio import Entrez

Entrez.email = "anybody.else@internet.com" # You should replace this with your e-mail address


#find out how many gene entries there are for Nrg1
handle = Entrez.egquery(term="Nrg1")
record = Entrez.read(handle)
handle.close()

# we can iterate through the record and only return the 'nucleotide' result
for row in record["eGQueryResult"]:
    if row["DbName"]=="gene":
        # print how many gene entries there are
        print("There are "+row["Count"]+" gene entries for the gene Nrg1")

# you might notice this is different to the web page result. if you click the gene link you will notice on
# the side box that details the query another term has been added to make sure the returned gene entries are
# live in the database. this excludes retired and/or redirected links and gives the true value
handle = Entrez.egquery(term="Nrg1 AND alive[prop]")
record = Entrez.read(handle)
handle.close()

# we can iterate through the record and only return the 'nucleotide' result
for row in record["eGQueryResult"]:
    if row["DbName"]=="gene":
        # print how many alive gene entries there are
        print("There are "+row["Count"]+" live gene entries for the gene Nrg1")

In [ ]:
#search for the gene accessions
handle = Entrez.esearch(db='gene',term="Nrg1[Gene] AND human[Organism]")
human_gene_ids = Entrez.read(handle)['IdList']
handle.close()

# #fetch the first gene entry - this is the summary you've found already
print("human gene id",human_gene_ids)
handle = Entrez.efetch(db='gene',id=human_gene_ids[0],retmode='text')
print(handle.read())
handle.close()

#OK so this is the XML data which is the full record as you've seen
handle = Entrez.efetch(db='gene',id=human_gene_ids,rettype='gb',retmode='xml')

#I'm going to use ElementTree to parse the XML, this is PAINFUL!
#You need to examine the structure of the XML to work out which element tags to use, I'm not sure there's an easier way!
import xml.etree.ElementTree as ET
tree = ET.parse(handle)
all_name_elements = tree.findall('.//Other-source')

print("GO annotations for the Gene\n")
for source in all_name_elements:
    source_ids = source.findall('.//Dbtag_db')
    for source_id in source_ids:
        if source_id.text == 'GO':
            GO_terms = source.findall('Other-source_anchor')
            for GO_term in GO_terms:
                print(GO_term.text)
            # print(ET.tostring(source))
        else:
            pass

handle.close()


### Challenge 2 - Human and Mouse Nrg1 Genes
Using either the Entrez website to search and/or using what you've learned about BioPython's abilities to query NCBI services retreive full-length human and mouse (RefSeq) gene entries for Nrg1.
- What are the accession numbers / ids of the Genbank records?
- How long are the Human and Mouse NRG1, Nrg1 proteins?
- How many nucleotide sequence differences are there between their longest CDs?
- How many protein sequence differences are there between their longest proteins?

In [ ]:
from Bio import SeqIO

#From above we can find the human and mouse versions the accession ids
handle = Entrez.esearch(db='gene',term="Nrg1[Gene] AND human[Organism]")
human_gene_ids = Entrez.read(handle)['IdList']
handle = Entrez.esearch(db='gene',term="Nrg1[Gene] AND mouse[Organism]")
mouse_gene_ids = Entrez.read(handle)['IdList']
handle.close()

#Accession Numbers of Gene Entries
print("human gene id",human_gene_ids)
print("mouse gene ids",mouse_gene_ids)

#convenience function to find the genomic sequence entries from a gene_id
def find_gene_sequence(gene_id):
    handle = Entrez.efetch(db='gene',id=gene_id,rettype='gb',retmode='xml')
    gene_entry = Entrez.read(handle)

    #Get the accession, start and stop locations from the genbank XML file
    accession = gene_entry[0]['Entrezgene_locus'][0]['Gene-commentary_accession']
    start = gene_entry[0]['Entrezgene_locus'][0]['Gene-commentary_seqs'][0]['Seq-loc_int']['Seq-interval']['Seq-interval_from']
    stop = gene_entry[0]['Entrezgene_locus'][0]['Gene-commentary_seqs'][0]['Seq-loc_int']['Seq-interval']['Seq-interval_to']

    #Retreive the annotated sequence and parse it for protein features
    handle = Entrez.efetch(db='nuccore',id=accession, seq_start=start, seq_stop=stop, rettype='gb', retmode='text')
    record = SeqIO.read(handle, "genbank")
    return(record)

#Get the gene records
human_record = find_gene_sequence(human_gene_ids[0])
mouse_record = find_gene_sequence(mouse_gene_ids[0])

# for convenience I've defined a function that allows me to pass any suitable record and find the longest
# protein sequence
def find_longest_protein(record):
    longest_protein_length = 0
    longest_cds = ''

    if record.features:
        for feature in record.features:
            #this tag identifies the CoDingSequences from the record
            if feature.type == "CDS":
                current_sequence = feature.location.extract(record).seq
                #check whether sequence is a multiple of three if not pad out end with Ns (optional)
                # translate will throw an error if the sequence is not a multiple of three
                if len(current_sequence) % 3 != 0:
                    current_sequence += 'N' * ((3 - len(current_sequence) % 3))
                #translate the current sequence into protein
                translation = current_sequence.translate()
                if len(translation) > longest_protein_length:
                    longest_protein_length = len(translation)
                    longest_cds = current_sequence
    #             print(feature.qualifiers["gene"],feature.qualifiers["protein_id"],len(translation))

    print("Longest Protein -",longest_protein_length,"amino acids\nCDS -",longest_cds,"\nProtein -",longest_cds.translate())
    handle.close()
    return(longest_cds)

# call the function to find the longest proteins for these genes
print("Human")
human_cd = find_longest_protein(human_record)
print("Mouse")
mouse_cd = find_longest_protein(mouse_record)

# The last two questions can be done online, but in order to do them programatically you need to be able to perform
# pairwise sequence alignment using python

In [ ]:
from Bio import pairwise2

#CDS
alignment = pairwise2.align.globalxx(human_cd,mouse_cd)

#the number of identical matches in the CDS alignment
aligned = alignment[0][2]

#work out the non-identical matches per sequence
print("Nucleotide")
print("Non-aligned human bases",int(len(human_cd)-aligned))
print("Non-aligned mouse bases",int(len(mouse_cd)-aligned))


#Proteins
alignment = pairwise2.align.globalxx(human_cd.translate(),mouse_cd.translate())

#the number of identical matches in the protein lignment
aligned = alignment[0][2]

#work out the non-identical matches per sequence
print("\nProtein")
print("Non-aligned human amino acids",int(len(human_cd.translate())-aligned))
print("Non-aligned mouse amino acids",int(len(mouse_cd.translate())-aligned))